In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"

# Gather prices

In [15]:
# item ID file
objects = pd.read_json('itemID.json',typ='series')

name = "Noxious scythe"

# group = "Airut bones"
group = name

class get_item:
    def __init__(self,name,graph='False'):
        self.name = name
        self.graph = graph
        self.ID = self.get_ID(name)
        self.item = requests.get("https://services.runescape.com/m=itemdb_rs/api/catalogue/detail.json?item={}".format(self.ID)).json()['item']
        self.type = self.item['type']
    def get_ID(self,name):
        for ID,item in objects.items():
            if item == name:
                return ID
    '''
    val can be:
        - current
        - today
        - day30 
        - day60
        - day90
        - day180
    '''
    def price(self,group,val='current',graph_period = 'daily'):
        prices = {}
        graphs = {}
        
        item_groups = []
        for ID,item in objects.items():
            if group in item:
                item_groups.append(item)
        if type(item_groups) == list:
            for item in item_groups:
                ID = self.get_ID(item)
                item_info = requests.get("https://services.runescape.com/m=itemdb_rs/api/catalogue/detail.json?item={}".format(ID)).json()['item'][val]
                if eval(self.graph) == True:
                    graph_info = requests.get("https://services.runescape.com/m=itemdb_rs/api/graph/{}.json".format(ID)).json()
                    graphs[item + ' graph'] = graph_info
                prices[item] = item_info
        else:
            item_info = requests.get("https://services.runescape.com/m=itemdb_rs/api/catalogue/detail.json?item={}".format(self.ID)).json()['item'][val]
        return prices,graphs
            

instance = get_item(name,graph='True')
instance_price = instance.price(group,'day180')
info = instance_price[1]
# display(pd.DataFrame(info['Noxious staff graph']))
# name = "Willow logs"
# r = requests.get("https://services.runescape.com/m=itemdb_rs/api/catalogue/detail.json?item={}".format(get_item_ID(name)))
print(instance.type)


Melee weapons - high level


In [3]:
print(instance.item)

{'icon': 'https://secure.runescape.com/m=itemdb_rs/1612796435832_obj_sprite.gif?id=28227', 'icon_large': 'https://secure.runescape.com/m=itemdb_rs/1612796435832_obj_big.gif?id=28227', 'id': 28227, 'type': 'Potions', 'typeIcon': 'https://www.runescape.com/img/categories/Potions', 'name': 'Super Saradomin brew flask (6)', 'description': '6 doses of super Saradomin brew in a flask.', 'current': {'trend': 'neutral', 'price': '209.2k'}, 'today': {'trend': 'neutral', 'price': 0}, 'members': 'true', 'day30': {'trend': 'positive', 'change': '+2.0%'}, 'day90': {'trend': 'positive', 'change': '+19.0%'}, 'day180': {'trend': 'positive', 'change': '+34.0%'}}


In [4]:
from datetime import datetime
import plotly.express as px

item = name

price_frame = pd.DataFrame(info[item + ' graph']['daily'],index=['price']).transpose().reset_index()
price_frame['index'] = price_frame['index'].apply(lambda x: datetime.fromtimestamp(int(x)/1000).strftime('%Y-%m-%d'))
price_frame.set_index('index',inplace=True)

In [10]:
import plotly.graph_objects as go

fig = go.Figure()

dfxp = pd.read_csv('dxp_weeks.csv')
for col in dfxp:
    dfxp[col] = dfxp[col].apply(lambda x: datetime.strptime(x, '%d %B %Y (%H:%M)').strftime('%Y-%m-%d'))

print(dfxp)
# subset_starting = {i:'red' for i in dfxp.Starting_Date.tolist()}
# subset_ending = {i:'red' for i in dfxp.Ending_Date.tolist()}
# print(subset_starting)
# subsets = subset_starting

prices = price_frame.reset_index().rename(columns={'index':'date'})

# prices['Colour'] = prices['date'].apply(lambda x: 'red' if x in subsets.keys() else 'blue')

window = 15

prices['average'] = prices.price.rolling(window).mean()


# fig.add_trace(go.Scatter(x=prices.date, y=prices.Colour,
#                     mode='markers',
#                     name='markers'))

fig.add_trace(go.Scatter(x=prices.date, y=prices.average,
                    mode='lines',
                    name='Running average over {} days'.format(window)))
fig.add_trace(go.Scatter(x=prices.date, y=prices.price,
                    mode='lines',
                    name='Price of {}'.format(item)))


for index,row in dfxp.iterrows():
    if row[0] in list(prices.date):
        fig.add_vrect(
            x0="{}".format(row[0]), x1="{}".format(row[1]),
            fillcolor="LightSalmon",
            layer="below"
        )

prices.set_index('date',inplace=True)
prices = prices.astype(object)

fig.show()

# display(prices)
# display(dfxp)



# fig = px.scatter(prices,title=item, template="simple_white",color='Colour')
# px.scatter(prices[['average','Colour']],title=item, template="simple_white",color='Colour')
# display(prices)

                Starting_Date                Ending_Date
0    25 December 2020 (00:00)     1 January 2021 (00:00)
1     6 November 2020 (12:00)   16 November 2020 (12:00)
2       7 August 2020 (12:00)     17 August 2020 (12:00)
3          8 May 2020 (12:00)        18 May 2020 (12:00)
4    21 February 2020 (12:00)       2 March 2020 (12:00)
5    22 November 2019 (12:00)    2 December 2019 (12:00)
6        26 July 2019 (12:00)       29 July 2019 (12:00)
7    22 February 2019 (12:00)   25 February 2019 (12:00)
8    16 November 2018 (12:00)   19 November 2018 (12:00)
9      31 August 2018 (12:00)   3 September 2018 (12:00)
10   23 February 2018 (12:00)   26 February 2018 (12:00)
11  15 September 2017 (12:00)  18 September 2017 (12:00)
12        19 May 2017 (12:00)        22 May 2017 (12:00)
13   30 December 2016 (12:00)     4 January 2017 (12:00)
14  23 September 2016 (12:00)  26 September 2016 (12:00)
15   26 February 2016 (12:00)   29 February 2016 (12:00)
16  25 September 2015 (12:00)  